In [69]:
import os.path
import random
import numpy as np
import cv2
import torch
import torch.utils.data as data

#load HMI-GL bmp
imgPI = r"C:\Users\donwu\Documents\github\RDN_HMI\RDN-Pytorch"

def GLBMP_np2Tensor(imgIn, imgTar):
    ts = (2, 0, 1)
    imgIn = torch.Tensor(imgIn.transpose(ts).astype(float)).mul_(1.0)
    imgTar = torch.Tensor(imgTar.transpose(ts).astype(float)).mul_(1.0)
    return imgIn, imgTar
#GLBMP_np2tensor(imgI, imgT)

In [70]:
#augment
def augment(imgIn, imgTar):
    if random.random() < 0.3:
        imgIn = imgIn[:, ::-1, :]
        imgTar = imgTar[:, ::-1, :]
    if random.random() < 0.3:
        imgIn = imgIn[::-1, :, :]
        imgTar = imgTar[::-1, :, :]
    return imgIn, imgTar

#augment(imgI, imgT)

In [76]:
def getPatch(imgIn, imgTar, scale):
    (ih, iw, c) = imgIn.shape
    print(imgIn.shape)
    (th, tw) = (scale * ih, scale * iw)
    tp = 10
    ip = tp // scale
    ix = random.randrange(0, iw - ip + 1)
    iy = random.randrange(0, ih - ip + 1)
    (tx, ty) = (scale * ix, scale * iy)
    imgIn = imgIn[iy:iy + ip, ix:ix + ip, :]
    imgTar = imgTar[ty:ty + tp, tx:tx + tp, :]
    return imgIn, imgTar

In [77]:
class HMIGLBMP():
    def __init__(self):        
        self.scale = 3
        apath = r"C:\Users\donwu\Documents\github\RDN_HMI\RDN-Pytorch\data\hmi"
        dirHR = 'train_gt'
        dirLR = 'train'
        self.dirIn = os.path.join(apath, dirLR)
        #print("traindata:" + self.dirIn)
        self.dirTar = os.path.join(apath, dirHR)
        self.fileList= os.listdir(self.dirIn)
        
        self.nTrain = len(self.fileList)
        #print(self.nTrain)
        
    def __getitem__(self, idx):        
        scale = self.scale
        nameIn, nameTar = self.getFileName(idx)
        imgIn = cv2.imread(nameIn)
        imgTar = cv2.imread(nameTar)
        imgIn, imgTar = getPatch(imgIn, imgTar,scale)
        #print(imgIn)
        imgIn, imgTar = augment(imgIn, imgTar)
        return GLBMP_np2Tensor(imgIn, imgTar)

    def __len__(self):
        return self.nTrain
        
    def getFileName(self, idx):
        name = self.fileList[idx]
        nameTar = os.path.join(self.dirTar, name)
        name = name[0:-4] + '.bmp'
        nameIn = os.path.join(self.dirIn, name)
        return nameIn, nameTar

In [78]:
#args = [batchSize=16, dataDir='./data/hmi/', decayType='step', epochs=100, finetuning=False, growthRate=32, load='demo_x3_RDN', lossType='L1\r', lr=0.0001, lrDecay=2000, model_name='RDN', nBlock=16, nChannel=3, nDenselayer=3, nFeat=64, nThreads=4, need_patch='True', patchSize=10, saveDir='.\result', scale=3]
HMIGLBMP()